In [144]:
import pandas as pd

In [145]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target = train['SalePrice']

In [146]:
train_original = train.copy()
test_original = test.copy()

In [147]:
train_predictors = train.drop(['Id', 'SalePrice'], axis= 1)
test_predictors = test.drop(['Id'], axis= 1)
len(train_predictors.columns)

79

In [148]:
miss_count = train_predictors.isnull().sum()
cols_with_missing_data = miss_count[miss_count > 0]
print(len(cols_with_missing_data))

19


In [149]:
high_cordinality_cols = [col for col in train_predictors.columns if train_predictors[col].dtype == 'object' and train_predictors[col].nunique() >= 10]
low_cordinality_cols = [col for col in train_predictors.columns if train_predictors[col].dtype == 'object' and train_predictors[col].nunique() < 10]
numeric_cols = [col for col in train_predictors.columns if train_predictors[col].dtype in ['int64', 'float64']]
print('Length of High Cordinal Object columns: ', len(high_cordnality_cols))
print('Length of Low Cordinal Object columns: ', len(low_cordinality_cols))
print('Length of Numeric columns: ', len(numeric_cols))

Length of High Cordinal Object columns:  3
Length of Low Cordinal Object columns:  40
Length of Numeric columns:  36


### Handling Missing Values

#### 1. Drop Columns/Rows with Missing Values

Imputation fills in the missing value with some number. The imputed value won't be exactly right in most cases, but it usually gives more accurate models than dropping the column entirely.

In [150]:
#train_without_missing_values = train.dropna(axis = 1)
#train_without_missing_values = train.dropna(axis = 0)
#test_without_missing_values = test.dropna(axis = 1)
#test_without_missing_values = test.dropna(axis = 0)

In [151]:
train_predictors = train_predictors.drop(high_cordinality_cols, axis= 1)
print('Length of Predictor columns after removing High Cordinality columns is: ', len(train_predictors.columns))
train_predictors.head()

Length of Predictor columns after removing High Cordinality columns is:  76


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


#### 2. Imputation

In [152]:
train_numeric_predictors = train_predictors[numeric_cols]
train_categorical_predictors = train_predictors[low_cordinality_cols]
train_categorical_predictors.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [153]:
print('Length of High Cordinal Object columns: ', len(train_numeric_predictors.columns))
print('Length of Low Cordinal Object columns: ', len(train_categorical_predictors.columns))

Length of High Cordinal Object columns:  36
Length of Low Cordinal Object columns:  40


In [154]:
from sklearn.preprocessing import Imputer

In [155]:
my_imputer = Imputer()
train_numeric_predictors.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


In [156]:
imputed_array_predictors = my_imputer.fit_transform(train_numeric_predictors)
train_imputed_numeric_predictors = pd.DataFrame(imputed_array_predictors, columns= train_numeric_predictors.columns)
train_imputed_numeric_predictors.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,608.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,642.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,836.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0


In [157]:
len(train_imputed_numeric_predictors.columns)

36

##### Process with Numerical Predictors here

In [158]:
train_hot_encoded_predictors = pd.get_dummies(train_categorical_predictors)
print('Number of columns in Hot encoded Dataframe: ', len(train_hot_encoded_predictors.columns))
train_hot_encoded_predictors.head()

Number of columns in Hot encoded Dataframe:  196


,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


In [159]:
train_predictors_final = train_imputed_numeric_predictors.join(train_hot_encoded_predictors)

miss_train_predictors = train_predictors_final.isnull().sum()
print(miss_train_predictors[miss_train_predictors > 0])

Series([], dtype: int64)


##### Fully Processessed

In [160]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [161]:
def get_mae(X, y):
    return -1 * cross_val_score(RandomForestRegressor(100), X, y, scoring= 'neg_mean_absolute_error').mean()

In [162]:
print('Mean absolute error when dropping Categorical variables', int(get_mae(train_imputed_numeric_predictors, target)))
print('Mean absolute error when including Categorical variables', int(get_mae(train_predictors_final, target)))

Mean absolute error when dropping Categorical variables 18095
Mean absolute error when including Categorical variables 17875


##### Ensure the test data is encoded in the same manner as the training data with the align command:

In [163]:
# Segmenting type of variables
test_high_cordinality_cols = [col for col in test_predictors.columns if test_predictors[col].dtype == 'object' and test_predictors[col].nunique() >= 10]
test_low_cordinality_cols = [col for col in test_predictors.columns if test_predictors[col].dtype == 'object' and test_predictors[col].nunique() < 10]
test_numeric_cols = [col for col in test_predictors.columns if test_predictors[col].dtype in ['int64', 'float64']]

# Dropping high cordinality columns
test_predictors = test_predictors.drop(high_cordinality_cols, axis= 1)
print('Length of Predictor columns after removing High Cordinality columns is: ', len(train_predictors.columns))

# Creating two different dataframes based on the process to be followed
test_numeric_predictors = test_predictors[test_numeric_cols]
test_categorical_predictors = test_predictors[test_low_cordinality_cols]
print('Length of Numerocal Predictor columns is: ', len(test_numeric_predictors.columns))
print('Length of Categorical Predictor columns is: ', len(test_categorical_predictors.columns))

Length of Predictor columns after removing High Cordinality columns is:  76
Length of Numerocal Predictor columns is:  36
Length of Categorical Predictor columns is:  40


In [164]:
# Processing with Numerical columns
test_imputed_array_predictors = my_imputer.fit_transform(test_numeric_predictors)
test_imputed_numerical_predictors = pd.DataFrame(test_imputed_array_predictors, columns= test_numeric_predictors.columns)
print('Number of columns in Imputed d Dataframe: ', len(test_imputed_numerical_predictors.columns))

Number of columns in Imputed d Dataframe:  36


In [165]:
# Hot Encoding for Categorical columns
test_hot_encoded_predictors = pd.get_dummies(test_categorical_predictors)
print('Number of columns in Hot encoded Dataframe: ', len(test_hot_encoded_predictors.columns))

Number of columns in Hot encoded Dataframe:  181


In [168]:
test_predictors_final = test_imputed_numerical_predictors.join(test_hot_encoded_predictors)

miss_test_predictors = test_predictors_final.isnull().sum()
print(miss_test_predictors[miss_test_predictors > 0])

Series([], dtype: int64)


So far, we have one-hot-encoded the training data. What about when we have multiple files (e.g. a test dataset, or some other data that you'd like to make predictions for)? Scikit-learn is sensitive to the ordering of columns, so if the training dataset and test datasets get misaligned, your results will be nonsense. This could happen if a categorical had a different number of unique values in the training data vs the test data.

In [167]:
final_train, final_test = train_predictors_final.align(test_predictors_final, join = 'left', axis = 1)

print('Length of columns in final train dataset ', len(final_train.columns))
print('Length of columns in final test dataset ', len(final_test.columns))

Length of columns in final train dataset  232
Length of columns in final test dataset  232


The align command makes sure the columns show up in the same order in both datasets (it uses column names to identify which columns line up in each dataset.)  The argument `join='left'` specifies that we will do the equivalent of SQL's _left join_.  That means, if there are ever columns that show up in one dataset and not the other, we will keep exactly the columns from our training data.  The argument `join='inner'` would do what SQL databases call an _inner join_, keeping only the columns showing up in both datasets.  That's also a sensible choice.
